# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## Install OpenPose

In [17]:
import getpass
import os

password = getpass.getpass()
command = "sudo -S apt-get update" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))
# command = "sudo su"
# os.system('echo %s | %s' % (password, command))

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88,7 kB]
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:3 http://ppa.launchpad.net/papirus/papirus/ubuntu bionic InRelease
Hit:4 http://bo.archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://packages.microsoft.com/repos/code stable InRelease
Get:6 http://bo.archive.ubuntu.com/ubuntu bionic-updates InRelease [88,7 kB]
Get:7 http://bo.archive.ubuntu.com/ubuntu bionic-backports InRelease [74,6 kB]
Fetched 252 kB in 9s (27,0 kB/s)
Reading package lists...


0

In [26]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  command = "sudo tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local"
  os.system('echo %s | %s' % (password, command))
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  command = "sudo sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt"
  os.system('echo %s | %s' % (password, command))
  
  # install system dependencies
  command = "sudo apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev"
  os.system('echo %s | %s' % (password, command))
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
CUDA_TOOLKIT_ROOT_DIR not found or specified
-- Could NOT find CUDA (missing: CUDA_TOOLKIT_ROOT_DIR CUDA_NVCC_EXECUTABLE CUDA_INCLUDE_DIRS CUDA_CUDART_LIBRARY) 
-- Building with CUDA.
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for p

## Detect poses on a test video

In [21]:
# !rm -rf youtube.mp4
# # download the youtube with the given ID
# !youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$(YOUTUBE_ID)
# cut the first 5 seconds
# !ffmpeg -y -loglevel info -i openpose/examples/media/video.avi -t 5 openpose/video.mp4
# detect poses on the these 5 seconds
# !rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video openpose/video.mp4 --write_json openpose/output/ --display 0  --write_video ./openpose/openpose.avi
# # convert the result into MP4

# !ffmpeg -y -loglevel info -i openpose/openpose.avi output.mp4

/bin/bash: ./build/examples/openpose/openpose.bin: No such file or directory


Finally, visualize the result:

In [7]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('openpose/video.mp4', width=960, height=720)